<a href="https://colab.research.google.com/github/SFIComplexityExplorer/Mesa-ABM-Tutorial/blob/main/Session_11_Traders_Move_part_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Winati Mutmainnah (D121221014)

# Import Dependencies

In [13]:
import mesa
import numpy as np
import math
import matplotlib.pyplot as plt

# Resource Classes

In [14]:
class Sugar(mesa.Agent):
    """
    Sugar:
    - contains an amount of sugar
    - grows 1 amount of sugar at each turn
    """

    def __init__(self, unique_id, model, pos, max_sugar):
        super().__init__(unique_id, model)
        self.pos = pos
        self.amount = max_sugar
        self.max_sugar = max_sugar

    def step(self):
        '''
        Sugar growth function, adds one unit of sugar each step until
        max amount
        '''
        self.amount = min([self.max_sugar, self.amount+1])

In [15]:
class Spice(mesa.Agent):
    """
    Spice:
    - contains an amount of spice
    - grows 1 amount of spice at each turn
    """

    def __init__(self, unique_id, model, pos, max_spice):
        super().__init__(unique_id, model)
        self.pos = pos
        self.amount = max_spice
        self.max_spice = max_spice

    def step(self):
        '''
        Spice growth function, adds one unit of spice each step until
        max amount
        '''
        self.amount = min([self.max_spice, self.amount+1])

# Trader Agent

In [16]:
class Trader(mesa.Agent):
    """
    Trader:
    - has a metabolism of sugar and spice
    - harvest and trade sugar and spice to survive
    """

    def __init__(self, unique_id, model, pos, moore=False, sugar=0,
                 spice=0, metabolism_sugar=0, metabolism_spice=0,
                 vision=0):
        super().__init__(unique_id, model)
        self.pos = pos
        self.moore = moore
        self.sugar = sugar
        self.spice = spice
        self.metabolism_sugar = metabolism_sugar
        self.metabolism_spice = metabolism_spice
        self.vision = vision

    def get_sugar(self, pos):
        '''
        used in self.get_sugar_amount()
        '''
        this_cell = self.model.grid.get_cell_list_contents(pos)
        for agent in this_cell:
            if isinstance(agent, Sugar):
                return agent
        return None

    def get_sugar_amount(self, pos):
        '''
        used in self.move() as part of self.calculate_welfare()
        '''
        sugar_patch = self.get_sugar(pos)
        if sugar_patch:
            print("Sugar amount at position {}: {}".format(pos, sugar_patch.amount))
            return sugar_patch.amount
        return 0

    def get_spice(self, pos):
        '''
        used in self.get_spice_amount()
        '''
        this_cell = self.model.grid.get_cell_list_contents(pos)
        for agent in this_cell:
            if isinstance(agent, Spice):
                return agent
        return None

    def get_spice_amount(self, pos):
        '''
        used in self.move() as part of self.calculate_welfare()
        '''
        spice_patch = self.get_spice(pos)
        if spice_patch:
            print("Spice amount at position {}: {}".format(pos, spice_patch.amount))
            return spice_patch.amount
        return 0

    def is_occupied_by_other(self, pos):
        '''
        helper function part 1 of self.move()
        '''
        if pos == self.pos:
            # agent's position is considered unoccupied as agent can stay there
            return False
        # get contents of each cell in neighborhood
        this_cell = self.model.grid.get_cell_list_contents(pos)
        for a in this_cell:
            # see if occupied by another agent
            if isinstance(a, Trader):
                return True
        return False

    def calculate_welfare(self, sugar, spice):
        '''
        helper function part 2 self.move()
        '''
        # calculate total metabolism
        m_total = self.metabolism_sugar + self.metabolism_spice
        # Cobb-Douglas functional form
        return sugar**(self.metabolism_sugar/m_total) * spice**(
            self.metabolism_spice/m_total)

    ######################################################################
    #                                                                    #
    #                      MAIN TRADE FUNCTIONS                          #
    #                                                                    #
    ######################################################################

    def move(self):
        '''
        Function for trader agent to identify optimal move for each step in 4 parts
        1 - identify all possible moves
        2 - determine which move maximizes welfare
        3 - find closest best option
        4 - move
        '''
        # 1. identify all possible moves
        neighbors = [i
                     for i in self.model.grid.get_neighborhood(
                       self.pos, self.moore, True, self.vision
                     ) if not self.is_occupied_by_other(i)]

        # 2. determine which move maximizes welfare
        welfares = [
            self.calculate_welfare(
                self.sugar + self.get_sugar_amount(pos),
                self.spice + self.get_spice_amount(pos))
            for pos in neighbors
        ]

        # For debugging: print the positions and their corresponding welfare
        for pos, welfare in zip(neighbors, welfares):
            print(f"Position {pos} has welfare {welfare}")

        # You may proceed to implement steps 3 and 4 as needed

# Model Class

In [17]:
class SugarscapeG1mt(mesa.Model):
    """
    Manager class to run Sugarscape with Traders
    """

    def __init__(self, width=50, height=50, initial_population=200,
                 endowment_min=25, endowment_max=50, metabolism_min=1,
                 metabolism_max=5, vision_min=1, vision_max=5):

        # Initialize the base Model class
        super().__init__()

        # Initiate width and height of sugarscape
        self.width = width
        self.height = height
        # Initiate population attributes
        self.initial_population = initial_population
        self.endowment_min = endowment_min
        self.endowment_max = endowment_max
        self.metabolism_min = metabolism_min
        self.metabolism_max = metabolism_max
        self.vision_min = vision_min
        self.vision_max = vision_max

        # initiate activation schedule
        self.schedule = mesa.time.RandomActivationByType(self)
        # initiate mesa grid class
        self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

        # read in landscape file from supplementary material
        sugar_distribution = np.genfromtxt("sugar-map.txt")
        spice_distribution = np.flip(sugar_distribution, 1)

        agent_id = 0
        for _, (x, y) in self.grid.coord_iter():
            max_sugar = sugar_distribution[x, y]
            if max_sugar > 0:
                sugar = Sugar(agent_id, self, (x, y), max_sugar)
                self.schedule.add(sugar)
                self.grid.place_agent(sugar, (x, y))
                agent_id += 1

            max_spice = spice_distribution[x, y]
            if max_spice > 0:
                spice = Spice(agent_id, self, (x, y), max_spice)
                self.schedule.add(spice)
                self.grid.place_agent(spice, (x, y))
                agent_id += 1

        for i in range(self.initial_population):
            # get agent position
            x = self.random.randrange(self.width)
            y = self.random.randrange(self.height)
            # give agents initial endowment
            sugar = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
            spice = int(self.random.uniform(self.endowment_min, self.endowment_max+1))
            # give agents initial metabolism
            metabolism_sugar = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
            metabolism_spice = int(self.random.uniform(self.metabolism_min, self.metabolism_max+1))
            # give agents vision
            vision = int(self.random.uniform(self.vision_min, self.vision_max+1))
            # create Trader object
            trader = Trader(agent_id,
                            self,
                            (x, y),
                            moore=False,
                            sugar=sugar,
                            spice=spice,
                            metabolism_sugar=metabolism_sugar,
                            metabolism_spice=metabolism_spice,
                            vision=vision)
            # place agent
            self.grid.place_agent(trader, (x, y))
            self.schedule.add(trader)
            agent_id += 1

    def step(self):
        '''
        Unique step function that does staged activation of sugar and spice
        and then randomly activates traders
        '''
        # step Sugar agents
        for sugar in self.schedule.agents_by_type[Sugar].values():
            sugar.step()

        # step Spice agents
        for spice in self.schedule.agents_by_type[Spice].values():
            spice.step()

        # step trader agents
        # to account for agent death and removal we need a separate data structure to iterate
        trader_shuffle = list(self.schedule.agents_by_type[Trader].values())
        self.random.shuffle(trader_shuffle)

        for agent in trader_shuffle:
            agent.move()

        self.schedule.steps += 1  # important for data collector to track number of steps

    def run_model(self, step_count=1000):
        for i in range(step_count):
            self.step()

# Run Sugarscape

In [18]:
model = SugarscapeG1mt()
model.run_model(step_count=1)

Sugar amount at position (4, 24): 2.0
Spice amount at position (4, 24): 2.0
Sugar amount at position (5, 23): 1.0
Spice amount at position (5, 23): 2.0
Sugar amount at position (5, 24): 2.0
Spice amount at position (5, 24): 2.0
Sugar amount at position (5, 25): 2.0
Spice amount at position (5, 25): 2.0
Sugar amount at position (6, 22): 1.0
Spice amount at position (6, 22): 2.0
Sugar amount at position (6, 23): 1.0
Spice amount at position (6, 23): 2.0
Sugar amount at position (6, 24): 2.0
Spice amount at position (6, 24): 2.0
Sugar amount at position (6, 25): 2.0
Spice amount at position (6, 25): 2.0
Sugar amount at position (6, 26): 2.0
Spice amount at position (6, 26): 1.0
Sugar amount at position (7, 23): 2.0
Spice amount at position (7, 23): 2.0
Sugar amount at position (7, 24): 2.0
Spice amount at position (7, 24): 2.0
Position (4, 24) has welfare 27.329300746880953
Position (5, 23) has welfare 26.650267695109957
Position (5, 24) has welfare 27.329300746880953
Position (5, 25) has

C:\Users\lenovo\AppData\Local\Temp\ipykernel_27152\2664322469.py:47: UserWarning: Agent 0 is being placed with
place_agent() despite already having the position (0, 0). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(spice, (x, y))
C:\Users\lenovo\AppData\Local\Temp\ipykernel_27152\2664322469.py:47: UserWarning: Agent 1 is being placed with
place_agent() despite already having the position (0, 1). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(spice, (x, y))
C:\Users\lenovo\AppData\Local\Temp\ipykernel_27152\2664322469.py:47: UserWarning: Agent 2 is being placed with
place_agent() despite already having the position (0, 2). In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.grid.place_agent(spice, (x, y))
C:\Users\lenovo\AppData\Local\Temp\ipykernel_27152\2